# TSA Chapter 10: GARCH(1,1) Volatility Forecast

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch10/TSA_ch10_garch_forecast/TSA_ch10_garch_forecast.ipynb)

This notebook demonstrates:
- Fitting a GARCH(1,1) model to Bitcoin returns and comparing rolling one-step volatility forecasts with realised volatility.

In [ ]:
!pip install yfinance arch matplotlib numpy pandas -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import yfinance as yf
from arch import arch_model
import os, warnings
warnings.filterwarnings('ignore')

In [ ]:
# Color scheme and style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12,
    'axes.labelsize': 10, 'xtick.labelsize': 9, 'ytick.labelsize': 9,
    'legend.fontsize': 9, 'figure.dpi': 150, 'lines.linewidth': 1.2,
    'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def legend_outside(ax, ncol=3, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
# Download Bitcoin data
btc = yf.download('BTC-USD', start='2019-01-01', progress=False)
if isinstance(btc.columns, pd.MultiIndex):
    btc.columns = btc.columns.get_level_values(0)
btc['Return'] = btc['Close'].pct_change() * 100
btc.dropna(inplace=True)
print(f'BTC data: {len(btc)} daily observations')

# Fit GARCH(1,1) rolling forecast
returns = btc['Return'].dropna()
n_train = int(len(returns) * 0.8)
test = returns.iloc[n_train:]

# Rolling one-step forecasts (subsample for speed)
step = max(1, len(test) // 60)
forecasts, f_idx = [], []
for i in range(0, len(test), step):
    window = returns.iloc[:n_train + i]
    mod = arch_model(window, vol='Garch', p=1, q=1, mean='Constant', dist='normal')
    r = mod.fit(disp='off', update_freq=0)
    fcast = r.forecast(horizon=1)
    forecasts.append(np.sqrt(fcast.variance.values[-1, 0]))
    f_idx.append(test.index[i])

realized_smooth = pd.Series(np.abs(test.values), index=test.index).rolling(20).mean()

fig, ax = plt.subplots(figsize=(8, 3.5))
ax.plot(f_idx, forecasts, color=COLORS['blue'], lw=0.8, label='GARCH(1,1) $\\hat{\\sigma}_t$')
ax.plot(test.index, realized_smooth, color=COLORS['red'], lw=0.8, alpha=0.7, label='$|r_t|$ (20-day MA)')
ax.set_ylabel('Volatility (%)')
ax.set_title('GARCH(1,1) Forecast vs Realised Volatility')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.setp(ax.get_xticklabels(), rotation=30, ha='right')
legend_outside(ax, ncol=2, y=-0.22)
fig.tight_layout()
save_chart(fig, 'garch_forecast')
plt.show()